# Symbolica exercise

This document contains a guided exercise for the use of Symbolica by studying an application in particle physics.

## Setup

Run the following cell for some basic setup. Make sure you are running in a virtual environment so that you have the rights to install Symbolica.

In [1]:
#%pip install --upgrade symbolica
from symbolica import *
from IPython.display import Markdown as md, display as dp

def display(self) -> None:
    dp(md(self.to_latex().replace('gamma', '\\gamma').replace('mu', '\\mu')))

def display_graph(self: Graph) -> None:
    dp(md('```mermaid\n' + self.to_mermaid().replace('[33m-[0m', '-').replace('[33m+[0m', '+') + '\n```'))


set_license_key('gcolab-demo-key-do-not-use-elsewhere')


## Graph generation

Generate graph by adding the proper vertex in the code below

In [2]:
ph, qu, m = S('ph', 'qu', 'm')

vertices = [((True, qu), (None, ph), (False, qu))]  # electron emits or absorbs a photon

graphs = Graph.generate([(1, (True, qu)), (2, (False, qu))], vertices, max_loops=1, max_bridges=0)

(graph, sym) = next(g for g in graphs.items())
display_graph(graph)
print("Symmetry factor:", sym)

```mermaid
graph TD;
  0["0"];
  1["0"];
  2["1"];
  3["2"];
  0 ---|"ph"| 1;
  0 -->|"qu"| 3;
  1 -->|"qu"| 0;
  2 -->|"qu"| 1;

```

Symmetry factor: 1


### Loop momentum basis

In [3]:
def spanning_tree(graph: Graph, parent: int, cur_node: int, parent_map: dict[int, tuple[int, int]]):
    if cur_node in parent_map:
        return
    parent_map[cur_node] = (parent, len(parent_map))
    for edge in graph.node(cur_node)[0]:
        (v1, v2,_,_) = graph.edge(edge)
        other = v1 if v2 == cur_node else v2
        spanning_tree(graph, cur_node, other, parent_map)

def get_loop_momenta(graph: Graph, parents: dict[int, tuple[int, int]]) -> list[int]:
    loop_mom_edges = []
    seen_edges = set()
    for edge_id in range(graph.num_edges()):
        (v1, v2 ,_, _) = graph.edge(edge_id)
        if parents[v1][0] == v2 or parents[v2][0] == v1:
            es = tuple(sorted((v1, v2)))
            if es in seen_edges:
                loop_mom_edges.append(edge_id)
            else:
                seen_edges.add(es)
        else:
            loop_mom_edges.append(edge_id)
    return loop_mom_edges

def assign_mom(graph: Graph, parents: dict[int, tuple[int, int]], cur: int, dest: int, mom: Expression):
    while cur != dest:
        spanning_tree_edge = next(ee for ee in range(graph.num_edges()) if sorted(graph.edge(ee)[:2]) == sorted((cur, parents[cur][0])))
        (v1, v2,dir,data) = graph.edge(spanning_tree_edge)
        if cur == v1:
            graph.set_edge_data(spanning_tree_edge, data + mom)
        else:
            graph.set_edge_data(spanning_tree_edge, data - mom)
        cur = parents[cur][0]

def assign_loop_mom(graph: Graph, parents: dict[int, tuple[int, int]], lm_edge: int, loop_mom_symbol: Expression):
    (v1, v2, _, data) = graph.edge(lm_edge)
    cur = v1 if parents[v1][1] > parents[v2][1] else v2
    dest = v1 if cur == v2 else v2
    graph.set_edge_data(lm_edge, data + loop_mom_symbol)
    assign_mom(graph, parents, cur, dest, loop_mom_symbol if cur == v2 else -loop_mom_symbol)

def assign_momenta(graph: Graph, loop_mom_symbol: Expression, ext_mom_symbol: Expression) -> Graph:
    mom_assignment = graph.__copy__()
    for edge_id in range(mom_assignment.num_edges()):
        mom_assignment.set_edge_data(edge_id, N(0))

    parents = {}
    # select one external as the root node
    root = next(n for n in range(mom_assignment.num_nodes()) if mom_assignment.node(n)[1] == 1)
    spanning_tree(mom_assignment, root, root, parents)

    lm = get_loop_momenta(mom_assignment, parents)

    for id, lm_edge in enumerate(lm):
        assign_loop_mom(mom_assignment, parents, lm_edge, loop_mom_symbol(id))

    # assign external momenta
    ext_count = 0
    for ext2 in range(graph.num_nodes()):
        if graph.node(ext2)[1] > 1:
            assign_mom(mom_assignment, parents, ext2, root, ext_mom_symbol(ext_count))
            ext_count += 1

    result = graph.__copy__()
    for edge_id in range(graph.num_edges()):
        result.set_edge_data(edge_id, S('edge')(result.edge(edge_id)[3], mom_assignment.edge(edge_id)[3]))
    return result

### Momentum assignment

Call `assign_momenta` on your graph and display it using `display_graph`

In [4]:
k, p = S('k', 'p')
graph = assign_momenta(graph, k, p)
display_graph(graph)

```mermaid
graph TD;
  0["0"];
  1["0"];
  2["1"];
  3["2"];
  0 ---|"edge(ph,k(0)+p(0))"| 1;
  0 -->|"edge(qu,-p(0))"| 3;
  1 -->|"edge(qu,k(0))"| 0;
  2 -->|"edge(qu,-p(0))"| 1;

```

## Feynman rules

Complete the code below that applies the Feynman rules to your expression called `graph`
- Use the edge index `i` to generate two dummy indices `mu(2*i)` and `mu(2*i + 1)`
- The expression should look like $\gamma(0, k(0), 3) \gamma(3, \mu(2), 4)  \cdots  prop(k, m^2)$
- Give the fermion a mass `m`

In [5]:
e = sym  # Initialize the expression

gamma, mu, prop, m = S('gamma', 'mu', 'prop', 'm')

# Loop over the nodes in the graph
for (n, (edges, _)) in enumerate(x for x in graph):
    if len(edges) == 1:  # external node
        continue
    
    # Find photon, fermion (q_bar), and the third edge (nq)
    photon = next(ei for ei in edges if graph.edge(ei)[3].contains(ph))  # photon edge id
    q_bar = next(ei for ei in edges if graph.edge(ei)[3].contains(qu) and graph.edge(ei)[1] == n)  # fermion edge id
    nq = next(x for x in edges if x not in [photon, q_bar])

    # Add the interaction (gamma matrix) for this vertex
    e = e * gamma(q_bar * 2 + 1, mu(photon * 2), nq * 2)

# Loop over the edges in the graph to account for the propagators
for i in range(graph.num_edges()):
    (v1, v2, dir, part) = graph.edge(i)
    mom = part[1]

    if part.contains(ph):  # Photon propagator
        if len(graph.node(v1)[0]) > 1 and len(graph.node(v2)[0]) > 1:
            e = e * prop(mom, 0)  # Photon propagator in the loop (massless)
    else:  # Fermion propagator
        e = e * (gamma(i * 2, mom, i * 2 + 1) + m * gamma(i * 2, i * 2 + 1)) * prop(mom, m**2)

# Expand the final expression for the amplitude
e = e.expand()

# Display the result
display(e)


$$\gamma\!\left(4,k\!\left(0\right),5\right) \gamma\!\left(5,\mu\!\left(0\right),2\right) \gamma\!\left(7,\mu\!\left(0\right),4\right) \gamma\!\left(2,-p\!\left(0\right),3\right) \gamma\!\left(6,-p\!\left(0\right),7\right) prop\!\left(k\!\left(0\right),m^{2}\right) prop\!\left(k\!\left(0\right)+p\!\left(0\right),0\right) prop\!\left(-p\!\left(0\right),m^{2}\right)^{2}+m \gamma\!\left(2,3\right) \gamma\!\left(4,k\!\left(0\right),5\right) \gamma\!\left(5,\mu\!\left(0\right),2\right) \gamma\!\left(7,\mu\!\left(0\right),4\right) \gamma\!\left(6,-p\!\left(0\right),7\right) prop\!\left(k\!\left(0\right),m^{2}\right) prop\!\left(k\!\left(0\right)+p\!\left(0\right),0\right) prop\!\left(-p\!\left(0\right),m^{2}\right)^{2}+m \gamma\!\left(4,5\right) \gamma\!\left(5,\mu\!\left(0\right),2\right) \gamma\!\left(7,\mu\!\left(0\right),4\right) \gamma\!\left(2,-p\!\left(0\right),3\right) \gamma\!\left(6,-p\!\left(0\right),7\right) prop\!\left(k\!\left(0\right),m^{2}\right) prop\!\left(k\!\left(0\right)+p\!\left(0\right),0\right) prop\!\left(-p\!\left(0\right),m^{2}\right)^{2}+m \gamma\!\left(6,7\right) \gamma\!\left(4,k\!\left(0\right),5\right) \gamma\!\left(5,\mu\!\left(0\right),2\right) \gamma\!\left(7,\mu\!\left(0\right),4\right) \gamma\!\left(2,-p\!\left(0\right),3\right) prop\!\left(k\!\left(0\right),m^{2}\right) prop\!\left(k\!\left(0\right)+p\!\left(0\right),0\right) prop\!\left(-p\!\left(0\right),m^{2}\right)^{2}+m^{2} \gamma\!\left(2,3\right) \gamma\!\left(4,5\right) \gamma\!\left(5,\mu\!\left(0\right),2\right) \gamma\!\left(7,\mu\!\left(0\right),4\right) \gamma\!\left(6,-p\!\left(0\right),7\right) prop\!\left(k\!\left(0\right),m^{2}\right) prop\!\left(k\!\left(0\right)+p\!\left(0\right),0\right) prop\!\left(-p\!\left(0\right),m^{2}\right)^{2}+m^{2} \gamma\!\left(2,3\right) \gamma\!\left(6,7\right) \gamma\!\left(4,k\!\left(0\right),5\right) \gamma\!\left(5,\mu\!\left(0\right),2\right) \gamma\!\left(7,\mu\!\left(0\right),4\right) prop\!\left(k\!\left(0\right),m^{2}\right) prop\!\left(k\!\left(0\right)+p\!\left(0\right),0\right) prop\!\left(-p\!\left(0\right),m^{2}\right)^{2}+m^{2} \gamma\!\left(4,5\right) \gamma\!\left(6,7\right) \gamma\!\left(5,\mu\!\left(0\right),2\right) \gamma\!\left(7,\mu\!\left(0\right),4\right) \gamma\!\left(2,-p\!\left(0\right),3\right) prop\!\left(k\!\left(0\right),m^{2}\right) prop\!\left(k\!\left(0\right)+p\!\left(0\right),0\right) prop\!\left(-p\!\left(0\right),m^{2}\right)^{2}+m^{3} \gamma\!\left(2,3\right) \gamma\!\left(4,5\right) \gamma\!\left(6,7\right) \gamma\!\left(5,\mu\!\left(0\right),2\right) \gamma\!\left(7,\mu\!\left(0\right),4\right) prop\!\left(k\!\left(0\right),m^{2}\right) prop\!\left(k\!\left(0\right)+p\!\left(0\right),0\right) prop\!\left(-p\!\left(0\right),m^{2}\right)^{2}$$

### Projection

Make the Feynman diagram a scalar by contracting the external indices
- The external edge indices are determined below

In [6]:
ext_edges = [i for i in range(graph.num_edges()) if len(graph.node(graph.edge(i)[0])[0]) == 1 or len(graph.node(graph.edge(i)[1])[0]) == 1]

g, x_, x___, y_, y___ = S('g', 'x_', 'x___', 'y_', 'y___')
e = e * g(mu(ext_edges[0] * 2), mu(ext_edges[1] * 2))
e = e.expand().replace(g(x_, y_)*gamma(x___, x_, y___), gamma(x___, y_, y___))
print(e.format(terms_on_new_line=True))

gamma(4,k(0),5)*gamma(5,mu(0),2)*gamma(7,mu(0),4)*gamma(2,-p(0),3)*gamma(6,-p(0),7)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))
+m*gamma(2,3)*gamma(4,k(0),5)*gamma(5,mu(0),2)*gamma(7,mu(0),4)*gamma(6,-p(0),7)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))
+m*gamma(4,5)*gamma(5,mu(0),2)*gamma(7,mu(0),4)*gamma(2,-p(0),3)*gamma(6,-p(0),7)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))
+m*gamma(6,7)*gamma(4,k(0),5)*gamma(5,mu(0),2)*gamma(7,mu(0),4)*gamma(2,-p(0),3)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))
+m²*gamma(2,3)*gamma(4,5)*gamma(5,mu(0),2)*gamma(7,mu(0),4)*gamma(6,-p(0),7)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))
+m²*gamma(2,3)*gamma(6,7)*gamma(4,k(0),5)*gamma(5,mu(0),2)*gamma(7,mu(0),4)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))
+m²*gamma(4,5)*gamma(6,7)*gamma(5,mu(0),2)*gamma(7,mu(0),4)*gamma(2,-p(0),3)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))
+

### Gamma trace

Contract the gamma matrices. You should have one gamma chain left after this operation.

In [7]:
e = e.replace(gamma(x___, x_)*gamma(x_, y___), gamma(x___, y___),repeat=True)
print(e)

gamma(6,-p(0),mu(0),k(0),mu(0),-p(0),3)*prop(k(0),m^2)*prop(k(0)+p(0),0)*prop(-p(0),m^2)^2*g(mu(2),mu(6))+m*gamma(6,mu(0),k(0),mu(0),-p(0),3)*prop(k(0),m^2)*prop(k(0)+p(0),0)*prop(-p(0),m^2)^2*g(mu(2),mu(6))+m*gamma(6,-p(0),mu(0),k(0),mu(0),3)*prop(k(0),m^2)*prop(k(0)+p(0),0)*prop(-p(0),m^2)^2*g(mu(2),mu(6))+m*gamma(6,-p(0),mu(0),mu(0),-p(0),3)*prop(k(0),m^2)*prop(k(0)+p(0),0)*prop(-p(0),m^2)^2*g(mu(2),mu(6))+m^2*gamma(6,mu(0),k(0),mu(0),3)*prop(k(0),m^2)*prop(k(0)+p(0),0)*prop(-p(0),m^2)^2*g(mu(2),mu(6))+m^2*gamma(6,mu(0),mu(0),-p(0),3)*prop(k(0),m^2)*prop(k(0)+p(0),0)*prop(-p(0),m^2)^2*g(mu(2),mu(6))+m^2*gamma(6,-p(0),mu(0),mu(0),3)*prop(k(0),m^2)*prop(k(0)+p(0),0)*prop(-p(0),m^2)^2*g(mu(2),mu(6))+m^3*gamma(6,mu(0),mu(0),3)*prop(k(0),m^2)*prop(k(0)+p(0),0)*prop(-p(0),m^2)^2*g(mu(2),mu(6))


Perform the fermion trace by calling `gamma_trace`.


In [8]:
# Perform the trace of a gamma matrix chain written as arguments of the function `tracer::trace()`.
# Returns contractions as `tracer::d`.
def gamma_trace(max_gamma_len: int) -> Transformer:
    T = Transformer()
    trace, x___, x_, y___, d = S('tracer::trace', 'tracer::x___', 'tracer::x_', 'tracer::y___', 'tracer::d')
    p_ = S(*['tracer::p' + str(i + 1) + '_' for i in range(max_gamma_len)])
    t = T
    for l in range(max_gamma_len, 0, -1):
        if l % 2 == 1:
            t = t.replace(trace(*p_[:l]), 0)
        else:
            t = t.chain(
                T.replace(trace(x___, x_, x_, y___),
                            d(x_, x_)*trace(x___, y___)),
                T.replace(trace(
                    *p_[:l]), sum((-1)**(k+1) * d(p_[0], p_[k]) * trace(*p_[1:k], *p_[k+1:l]) for k in range(1, l))).expand())
    return t.replace(trace(), 4)

In [9]:
trace = S("tracer::trace")

e = e.replace(gamma(x_, y___, x_), trace(y___))
traced = gamma_trace(10)(e)
print(traced.format(terms_on_new_line=True))


gamma(6,-p(0),mu(0),k(0),mu(0),-p(0),3)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))
+m*gamma(6,mu(0),k(0),mu(0),-p(0),3)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))
+m*gamma(6,-p(0),mu(0),k(0),mu(0),3)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))
+m*gamma(6,-p(0),mu(0),mu(0),-p(0),3)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))
+m²*gamma(6,mu(0),k(0),mu(0),3)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))
+m²*gamma(6,mu(0),mu(0),-p(0),3)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))
+m²*gamma(6,-p(0),mu(0),mu(0),3)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))
+m³*gamma(6,mu(0),mu(0),3)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))


Contract indices with pattern matching, making sure that $g^{\mu\mu} = D = 4-2\epsilon$

In [10]:
ep, td = S('ep', 'tracer::d')
d = S('d', is_symmetric=True, is_linear=True)

contract = traced.replace(td(x___), d(x___)).expand().replace(d(mu(x_),y_)*d(mu(x_), x___), d(y_, x___), repeat=True)
contract = contract.replace(d(mu(x_), x___)**2, d(x___, x___), repeat=True)
contract = contract.replace(d(mu(x_), mu(x_)), 4-2*ep)
print(contract.format(terms_on_new_line=True))

gamma(6,-p(0),mu(0),k(0),mu(0),-p(0),3)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))
+m*gamma(6,mu(0),k(0),mu(0),-p(0),3)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))
+m*gamma(6,-p(0),mu(0),k(0),mu(0),3)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))
+m*gamma(6,-p(0),mu(0),mu(0),-p(0),3)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))
+m²*gamma(6,mu(0),k(0),mu(0),3)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))
+m²*gamma(6,mu(0),mu(0),-p(0),3)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))
+m²*gamma(6,-p(0),mu(0),mu(0),3)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))
+m³*gamma(6,mu(0),mu(0),3)*prop(k(0),m²)*prop(k(0)+p(0),0)*prop(-p(0),m²)²*g(mu(2),mu(6))
